In [1]:
using Distributed
@everywhere begin 
    using ParallelTemperingMonteCarlo
    using DelimitedFiles
    using BenchmarkTools
end

In [2]:
using Random,Plots

#set random seed - for reproducibility
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details
mc_cycles = 100000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust, eq_percentage = 1)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=12.0*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)



Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[], Float64[])

In [3]:
ptmc_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false,save=true)
#plot(temp.t_grid,results.heat_cap)

# data = [results.en_histogram[i] for i in 1:n_traj]
# plot(data)

Total number of moves per MC cycle: 13



equilibration done


MC loop done.


[0.0003274332206663901, 0.00034097451808384846, 0.00035244948549771713, 0.00038565890500638536, 0.0003980335513622917, 0.00041254941004158604, 0.0004387579945671251, 0.00046105266182069345, 0.00048549186855110376, 0.0005021570901080064, 0.0005451170846650248, 0.0005775643450812362, 0.0006458749223787228, 0.0006770807764240569, 0.00071711013573714, 0.0008368237946829951, 0.0009831458227094459, 0.0011601596513707078, 0.0013070362090407607, 0.0017146656421000697, 0.0019621319526630914, 0.0030365686353476936, 0.0036432380891126223, 0.003581044352305293, 0.003610675334464941, 0.0032390163705791784, 0.0028215089701653243, 0.0028292964927889848, 0.003584854417093773, 0.003757680935178654, 0.004212266642843023, 0.004559761171251476]
done


Testing better mc_cycles

In [5]:
@everywhere function parallelmc_cycle!(mc_states, move_strat, mc_params, pot, ensemble, n_steps, a, v, r)
    #perform one MC cycle
    Threads.@threads for i_traj = 1:mc_params.n_traj
        for i_step = 1:n_steps
            #mc_states[i_traj] = mc_step!(type_moves[ran][2], type_moves[ran][1], mc_states[i_traj], ran, pot, ensemble)
            @inbounds mc_states[i_traj] = mc_step!(mc_states[i_traj], pot, ensemble, a, v, r)
        end
        #push!(mc_states[i_traj].ham, mc_states[i_traj].en_tot) #to build up ham vector of sampled energies
    end
    #parallel tempering
    if rand() < 0.1 #attempt to exchange trajectories
        n_exc = rand(1:mc_params.n_traj-1)
        mc_states[n_exc].count_exc[1] += 1
        mc_states[n_exc+1].count_exc[1] += 1
        exc_acc = exc_acceptance(mc_states[n_exc].beta, mc_states[n_exc+1].beta, mc_states[n_exc].en_tot,  mc_states[n_exc+1].en_tot)
        if exc_acc > rand()
            mc_states[n_exc].count_exc[2] += 1
            mc_states[n_exc+1].count_exc[2] += 1
            mc_states[n_exc], mc_states[n_exc+1] = exc_trajectories!(mc_states[n_exc], mc_states[n_exc+1])
        end
    end
    return mc_states
end

@everywhere function paralleltest(var)
    if var == -1
        for i in 1:100
            mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    elseif var == 1
        for i in 1:100
            parallelmc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    end
end


In [6]:
mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time1 = @benchmark paralleltest(-1)

mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time3 = @benchmark paralleltest(1)
println("no parallel = ",time1," parallelcycle = ",time3)

no parallel = Trial

(1.158 ms) parallelcycle = Trial(805.581 μs)


In [7]:
@everywhere function paralleltest(var)
    if var == -1
        for i in 1:1000
            mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    elseif var == 1
        for i in 1:1000
            parallelmc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    end
end

mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time1 = @benchmark paralleltest(-1)

mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time3 = @benchmark paralleltest(1)

println("no parallel = ",time1," parallelcycle = ",time3)

no parallel = Trial(11.970 ms) parallelcycle = Trial(9.125 ms)


In [10]:
@everywhere function parallelptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,n_steps ,a ,v ,r, save_ham, save, i ;delta_en=0. )
    #perform one MC cycle
    Threads.@threads for i_traj = 1:mc_params.n_traj
        for i_step = 1:n_steps
            #mc_states[i_traj] = mc_step!(type_moves[ran][2], type_moves[ran][1], mc_states[i_traj], ran, pot, ensemble)
            @inbounds mc_states[i_traj] = mc_step!(mc_states[i_traj], pot, ensemble, a, v, r)
        end
        #push!(mc_states[i_traj].ham, mc_states[i_traj].en_tot) #to build up ham vector of sampled energies
    end
    #parallel tempering
    if rand() < 0.1 #attempt to exchange trajectories
        n_exc = rand(1:mc_params.n_traj-1)
        mc_states[n_exc].count_exc[1] += 1
        mc_states[n_exc+1].count_exc[1] += 1
        exc_acc = exc_acceptance(mc_states[n_exc].beta, mc_states[n_exc+1].beta, mc_states[n_exc].en_tot,  mc_states[n_exc+1].en_tot)
        if exc_acc > rand()
            mc_states[n_exc].count_exc[2] += 1
            mc_states[n_exc+1].count_exc[2] += 1
            mc_states[n_exc], mc_states[n_exc+1] = exc_trajectories!(mc_states[n_exc], mc_states[n_exc+1])
        end
    end
    sampling_step!(mc_params,mc_states,i,save_ham)

    if save_ham == false
        updatehistogram!(mc_params,mc_states,results,delta_en,fullham=save_ham)
    end

    #step adjustment
    if rem(i, mc_params.n_adjust) == 0
        for i_traj = 1:mc_params.n_traj
            update_max_stepsize!(mc_states[i_traj], mc_params.n_adjust, a, v, r)
        end 
    end

    if save == true
        if rem(i,1000) == 0
            save_states(mc_params,mc_states,i)
            if save_ham == false
                save_results(results)
            end
        end
    end
    
end

@everywhere function ptmc_threaded_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham::Bool = true, save::Bool=true, restart::Bool=false,restartindex::Int64=0)
    #restart isn't compatible with saving the hamiltonian at the moment

    if restart == true
        save_ham = false
    end
    
    if save_ham == false
        if restart == false
            #If we do not save the hamiltonian we still need to store the E, E**2 terms at each cycle
            for i_traj = 1:mc_params.n_traj
                push!(mc_states[i_traj].ham, 0)
                push!(mc_states[i_traj].ham, 0)
            end
        end
    end

    a = atom_move_frequency(move_strat)
    v = vol_move_frequency(move_strat)
    r = rot_move_frequency(move_strat)
    #number of MC steps per MC cycle
    n_steps = a + v + r

    println("Total number of moves per MC cycle: ", n_steps)
    println()

    
    #equilibration cycle
    if restart == false
        #this initialises the max and min energies for histograms
        if save_ham == false
            ebounds = [100. , -100.] #emin,emax
        end
        
         for i = 1:mc_params.eq_cycles
            @inbounds mc_states = mc_cycle!(mc_states, move_strat, mc_params, pot, ensemble, n_steps, a, v, r)
            #verbose way to save the highest and lowest energies

            if save_ham == false
                for i_traj = 1:mc_params.n_traj
                    if mc_states[i_traj].en_tot < ebounds[1]
                        ebounds[1] = mc_states[i_traj].en_tot
                    end

                    if mc_states[i_traj].en_tot > ebounds[2]
                        ebounds[2] = mc_states[i_traj].en_tot
                    end
                end
            end
            #update stepsizes
            if rem(i, mc_params.n_adjust) == 0
                for i_traj = 1:mc_params.n_traj
                    update_max_stepsize!(mc_states[i_traj], mc_params.n_adjust, a, v, r)
                end 
            end
        end
    #re-set counter variables to zero
        for i_traj = 1:mc_params.n_traj
            mc_states[i_traj].count_atom = [0, 0]
            mc_states[i_traj].count_vol = [0, 0]
            mc_states[i_traj].count_rot = [0, 0]
            mc_states[i_traj].count_exc = [0, 0]
        end
        #initialise histogram for non-saving hamiltonian 
        if save_ham == false
            delta_en = initialise_histograms!(mc_params,mc_states,results, full_ham=false,e_bounds=ebounds)
        end

        println("equilibration done")
        

        if save == true
            save_states(mc_params,mc_states,0)
        end
    end


    #main MC loop

    if restart == false

        Threads.@threads for i = 1:mc_params.mc_cycles
            if save_ham == false
                parallelptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,n_steps ,a ,v ,r, save_ham, save, i;delta_en=delta_en)
            else
                parallelptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,n_steps ,a ,v ,r, save_ham, save, i)
            end
            
        end

    else #if restarting

        for i = restartindex:mc_params.mc_cycles
            if save_ham == false
                parallelptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,n_steps ,a ,v ,r, save_ham, save, i;delta_en=delta_en)
            else
                parallelptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,n_steps ,a ,v ,r, save_ham, save, i)
            end
            
        end 
    end
    println("MC loop done.")
    #Evaluation
    #average energy
    n_sample = mc_params.mc_cycles / mc_params.mc_sample

    if save_ham == true
        en_avg = [sum(mc_states[i_traj].ham) / n_sample for i_traj in 1:mc_params.n_traj] #floor(mc_cycles/mc_sample)
        en2_avg = [sum(mc_states[i_traj].ham .* mc_states[i_traj].ham) / n_sample for i_traj in 1:mc_params.n_traj]
    else
        en_avg = [mc_states[i_traj].ham[1] / n_sample  for i_traj in 1:mc_params.n_traj]
        en2_avg = [mc_states[i_traj].ham[2] / n_sample  for i_traj in 1:mc_params.n_traj]
    end


    results.en_avg = en_avg

    #heat capacity
    results.heat_cap = [(en2_avg[i]-en_avg[i]^2) * mc_states[i].beta for i in 1:mc_params.n_traj]

    #acceptance statistics
    results.count_stat_atom = [mc_states[i_traj].count_atom[1] / (mc_params.n_atoms * mc_params.mc_cycles) for i_traj in 1:mc_params.n_traj]
    results.count_stat_exc = [mc_states[i_traj].count_exc[2] / mc_states[i_traj].count_exc[1] for i_traj in 1:mc_params.n_traj]

    println(results.heat_cap)

    #energy histograms
    if save_ham == true
        # T = typeof(mc_states[1].ham[1])
        delta_en= initialise_histograms!(mc_params,mc_states,results)
        updatehistogram!(mc_params,mc_states,results,delta_en)
    
    end
    #     for i_traj in 1:mc_params.n_traj
    #         hist = zeros(results.n_bin)#EnHist(results.n_bin, global_en_min, global_en_max)
    #         for en in mc_states[i_traj].ham
    #             index = floor(Int,(en - global_en_min) / delta_en) + 1
    #             hist[index] += 1
    #         end
    #         push!(results.en_histogram, hist)
    #     end
    # end

    #TO DO
    # volume (NPT ensemble),rot moves ...
    # move boundary condition from config to mc_params?
    # rdfs

    println("done")
    return 
end

In [11]:
@benchmark begin
    ptmc_threaded_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false,save=true)
end

UndefVarError: UndefVarError: mc_states not defined

In [8]:
using Random
@everywhere import ..MCRun: initialise_histograms!,updatehistogram!,update_max_stepsize!,sampling_step!,save_results,save_states
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 50.
tf = 160.
n_traj = 16

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details
mc_cycles = 1000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

runnerdir = "/home/ghun245/RuNNer-master/Brass_potential/"
atomtype="Cu"
pot = AbstractMLPotential(runnerdir,atomtype)

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust, eq_percentage = 1)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
ico_13  = [[-0.0000000049,       -0.0000000044,       -0.0000000033],
[-0.0000007312,       -0.0000000014,        0.6554619119],
 [0.1811648930,      -0.5575692094,        0.2931316798],
[-0.4742970242,       -0.3445967289,        0.2931309525],
[-0.4742970303,        0.3445967144,        0.2931309494],
 [0.1811648830,        0.5575692066,        0.2931316748],
 [0.5862626299,        0.0000000022,        0.2931321262],
[-0.1811648928,       -0.5575692153,       -0.2931316813],
[-0.5862626397,       -0.0000000109,       -0.2931321327],
[-0.1811649028,        0.5575692007,       -0.2931316863],
 [0.4742970144,        0.3445967202,       -0.2931309590],
 [0.4742970205,       -0.3445967231,       -0.2931309559],
 [0.0000007214,       -0.0000000073,       -0.6554619185]]

#convert to Bohr
nmtobohr = 18.8973
copperconstant = 0.36258*nmtobohr
pos_cu13 = copperconstant*ico_13*1.5
AtoBohr = 1.8897259886


#boundary conditions 
bc_cu13 = SphericalBC(radius=12.0*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_cu13, bc_cu13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)

cd("/home/ghun245/ParallelTemperingMonteCarlo.jl/scripts")



 -------------------------------------------------------------
 ---------------------- Welcome to the -----------------------
     RuNNer Neural Network Energy Representation - RuNNer     
 ----------  (c) 2008-2022 Prof. Dr. Joerg Behler   ----------
 ----------  Georg-August-Universitaet Goettingen   ----------
 ----------           Theoretische Chemie           ----------
 ----------              Tammannstr. 6              ----------
 ----------        37077 Goettingen, Germany        ----------
 -------------------------------------------------------------
 -------------------------------------------------------------
  This program is free software: you can redistribute it and/or modify it
  under the terms of the GNU General Public License as published by the 
  Free Software Foundation, either version 3 of the License, or 
  (at your option) any later version.
    
  This program is distributed in the hope that it will be useful, but 
  WITHOUT ANY WARRANTY; without even the imp

   16 Cu  3  Zn Zn   0.000100  -1.000000   2.000000  11.338000
   17 Cu  3  Cu Cu   0.003000  -1.000000   1.000000  11.338000
   18 Cu  3  Cu Zn   0.003000  -1.000000   1.000000  11.338000
   19 Cu  3  Zn Zn   0.003000  -1.000000   1.000000  11.338000
   20 Cu  3  Cu Cu   0.003000  -1.000000   2.000000  11.338000
   21 Cu  3  Cu Zn   0.003000  -1.000000   2.000000  11.338000
   22 Cu  3  Zn Zn   0.003000  -1.000000   2.000000  11.338000
   23 Cu  3  Cu Cu   0.008000  -1.000000   1.000000  11.338000
   24 Cu  3  Cu Zn   0.008000  -1.000000   1.000000  11.338000
   25 Cu  3  Zn Zn   0.008000  -1.000000   1.000000  11.338000
   26 Cu  3  Cu Cu   0.008000  -1.000000   2.000000  11.338000
   27 Cu  3  Cu Zn   0.008000  -1.000000   2.000000  11.338000
   28 Cu  3  Zn Zn   0.008000  -1.000000   2.000000  11.338000
   29 Cu  3  Cu Cu   0.008000   1.000000   2.000000  11.338000
   30 Cu  3  Cu Zn   0.008000   1.000000   2.000000  11.338000
   31 Cu  3  Zn Zn   0.008000   1.000000   2.000000  11

 -------------------------------------------------------------
 Prediction Mode for all configurations
 -------------------------------------------------------------
 NNconfiguration:         1    Number of atoms:        13
 -------------------------------------------------------------
 begin
 atom      -0.00000005     -0.00000005     -0.00000003 Cu       0.00000000      0.00000000      0.00000000      0.00000000      0.00000000
 atom      -0.00000752     -0.00000001      6.73662421 Cu       0.00000000      0.00000000      0.00000000      0.00000000      0.00000000
 atom       1.86195381     -5.73051487      3.01271201 Cu       0.00000000      0.00000000      0.00000000      0.00000000      0.00000000
 atom      -4.87467046     -3.54165303      3.01270453 Cu       0.00000000      0.00000000      0.00000000      0.00000000      0.00000000
 atom      -4.87467052      3.54165288      3.01270450 Cu       0.00000000      0.00000000      0.00000000      0.00000000      0.00000000
 atom      

  37     0.00000000    0.87061230    0.03741902    0.87061230
  38     0.00000000    0.47723069    0.06422632    0.47723069
  39     0.00000000    0.27985557    0.03953078    0.27985557
  40     0.00000000    0.27092480    0.01072309    0.27092480
  41     0.00000000    0.08640216    0.00696992    0.08640216
  42     0.00000000    0.05758129    0.00382445    0.05758129
  43     0.00000000    0.04070981    0.00145938    0.04070981
  44     0.00000000    1.38645352    0.24927559    1.38645352
  45     0.00000000    0.67826645    0.15298757    0.67826645
  46     0.00000000    0.91198144    0.03728753    0.91198144
  47     0.00000000    3.47921965    0.71906469    3.47921965
  48     0.00000000    1.67321777    0.46299762    1.67321777
  49     0.00000000    2.54640219    0.10558866    2.54640219
  50     0.00000000    2.63729200    0.55464545    2.63729200
  51     0.00000000    1.29060244    0.36103177    1.29060244
  52     0.00000000    1.96372796    0.08130576    1.96372796
  53    

 Cutoff_alpha for inner cutoff radius is              0.000
 -------------------------------------------------------------
 Short range NN specifications:
 -------------------------------------------------------------
 global hidden layers short range NN                      2
 global nodes hidden layers short NN                20   20
 global activation functions short                      ttl
 -------------------------------------------------------------
 Parameters for symmetry function generation: short range part:
 -------------------------------------------------------------
 using forces for fitting                                T
 using atomic energies for fitting                       F
 -------------------------------------------------------------
 Options for prediction mode:
 -------------------------------------------------------------
 rescale symmetry functions                              T
 remove CMS from symmetry functions                      F
 rescale atomic char

...

 General input parameters:
 -------------------------------------------------------------
 NNP generation                                           2
 Short range NN is on
 Electrostatic NN is off
 vdW corrections switched off
 -------------------------------------------------------------
 RuNNer nn_type_short                                     1
 RuNNer is started in mode for prediction (3)
 debugging mode is                                       F
 parallelization mode                                     1
 enable detailed time measurement                        F
 using symmetry function groups                          F
 silent mode                                             F
 NN force check                                          F
 number of elements                                       2
 elements (sorted):
 29 Cu
 30 Zn
 seed for random number generator                      1000
 random number generator type                             5
 remove free atom reference energie

   74 Cu  3  Cu Cu   0.045000   1.000000  16.000000  11.338000
   75 Cu  3  Cu Zn   0.045000   1.000000  16.000000  11.338000
   76 Cu  3  Zn Zn   0.045000   1.000000  16.000000  11.338000
   77 Cu  3  Cu Cu   0.080000   1.000000   1.000000  11.338000
   78 Cu  3  Cu Zn   0.080000   1.000000   1.000000  11.338000
   79 Cu  3  Zn Zn   0.080000   1.000000   1.000000  11.338000
   80 Cu  3  Cu Cu   0.080000  -1.000000   2.000000  11.338000
   81 Cu  3  Cu Zn   0.080000  -1.000000   2.000000  11.338000
   82 Cu  3  Zn Zn   0.080000  -1.000000   2.000000  11.338000
   83 Cu  3  Cu Cu   0.080000  -1.000000   4.000000  11.338000
   84 Cu  3  Cu Zn   0.080000  -1.000000   4.000000  11.338000
   85 Cu  3  Zn Zn   0.080000  -1.000000   4.000000  11.338000
   86 Cu  3  Cu Cu   0.080000   1.000000   4.000000  11.338000
   87 Cu  3  Cu Zn   0.080000   1.000000   4.000000  11.338000
   88 Cu  3  Zn Zn   0.080000   1.000000   4.000000  11.338000
 ------------------------------------------------------

  22     0.00000000    2.63292295    0.07998148    2.63292295
  23     0.00036475    6.84954457    2.55727811    6.84917982
  24     0.00000000    3.37663635    0.60046955    3.37663635
  25     0.00000000    3.60734063    0.11012497    3.60734063
  26     0.00004682    2.87878694    1.00202413    2.87874011
  27     0.00000000    1.57328033    0.23637910    1.57328033
  28     0.00000000    1.54989710    0.04567094    1.54989710
  29     0.00330383   14.20003263    5.51399542   14.19672880
  30     0.00000000    6.07940896    1.27630877    6.07940896
  31     0.00000000    6.17826499    0.20316295    6.17826499
  32     0.00081609    9.46250663    3.58082950    9.46169054
  33     0.00000000    4.38136788    0.83294204    4.38136788
  34     0.00000000    4.39433923    0.13245838    4.39433923
  35     0.00000924    1.43539576    0.47859750    1.43538652
  36     0.00000000    0.82366127    0.11263600    0.82366127
  37     0.00000000    0.76031772    0.02156855    0.76031772
  38    

 -------------------------------------------------------------
 NN sum of free atom energies, short range and electrostatic energy for configuration         1
 NN sum of free atom energies         0.00000000 Ha
 NN short range energy               -0.51488321 Ha
 NN electrostatic energy              0.00000000 Ha
 NN vdw dispersion energy             0.00000000 Ha
 NNenergy                            -0.51488321 Ha
 -------------------------------------------------------------
 NN atomenergies for configuration            1
NNatomenergy      1 Cu      -0.065355
NNatomenergy      2 Cu      -0.037461
NNatomenergy      3 Cu      -0.037461
NNatomenergy      4 Cu      -0.037461
NNatomenergy      5 Cu      -0.037461
NNatomenergy      6 Cu      -0.037461
NNatomenergy      7 Cu      -0.037461
NNatomenergy      8 Cu      -0.037461
NNatomenergy      9 Cu      -0.037461
NNatomenergy     10 Cu      -0.037461
NNatomenergy     11 Cu      -0.037461
NNatomenergy     12 Cu      -0.037461
NNatomenergy  

Excessive output truncated after 525603 bytes.

In [13]:
function init_parallel_RuNNer(pot::AbstractMLPotential; n_threads = Threads.nthreads())
    if isdir("pots") == true
        rm("pots",recursive = true)
    end
    potential_directory = mkdir("$(pwd())/pots")
    potential_vector = []
    for i = 1:n_threads
        # n_threads potentials
        cp(pot.dir,"$(potential_directory)/potential$i")
        pot_i = AbstractMLPotential("$(potential_directory)/potential$i/",pot.atomtype)
        push!(potential_vector,pot_i)
    end
        
    return potential_vector        
end

function parallel_equilibration(mc_states,move_strat,mc_params,pot,ensemble,results)
    parallel_states = []

    pot_vector = init_parallel_RuNNer(pot) #initialise state and potentials

    n_threads = Threads.nthreads()
    sample_index = Int64(floor(mc_params.eq_cycles / n_threads)) #number of eq cycles per thread

    a = atom_move_frequency(move_strat)
    v = vol_move_frequency(move_strat)
    r = rot_move_frequency(move_strat)
    #number of MC steps per MC cycle
    n_steps = a + v + r

    println("Total number of moves per MC cycle: ", n_steps)
    println()

    ebounds = [100. , -100.]

    for i_thread = 1:n_threads
        for i_eq = 1:sample_index
            
            i = i_thread*sample_index + i_eq

            mc_states = mc_cycle!(mc_states, move_strat, mc_params, pot_vector[i_thread], ensemble, n_steps, a, v, r)#mc cycle
            
            for i_traj = 1:mc_params.n_traj#check energy bounds
                if mc_states[i_traj].en_tot < ebounds[1]
                    ebounds[1] = mc_states[i_traj].en_tot
                end

                if mc_states[i_traj].en_tot > ebounds[2]
                    ebounds[2] = mc_states[i_traj].en_tot
                end

            end

            if rem(i, mc_params.n_adjust) == 0 #adjust stepsize
                for i_traj = 1:mc_params.n_traj
                    update_max_stepsize!(mc_states[i_traj], mc_params.n_adjust, a, v, r)
                end 
            end

        end
        
        states_vec = [MCState(mc_states[i_traj].temp,mc_states[i_traj].beta,mc_states[i_traj].config,pot_vector[i_thread];max_displ = mc_states[i_traj].max_displ ) for i_traj in 1:mc_params.n_traj] #initialise a new mc_states vector based on current state

        for i_traj = 1:mc_params.n_traj
            push!(states_vec[i_traj].ham, 0)
            push!(states_vec[i_traj].ham, 0)
        end

        push!(parallel_states,states_vec) #add to vector of parallel states


    end
    
    

    delta_en = initialise_histograms!(mc_params,mc_states,results, full_ham=false,e_bounds=ebounds) #start histogram

    println("equilibration done")
    
    return parallel_states,pot_vector,a,v,r,delta_en,n_threads

end

parallel_equilibration (generic function with 1 method)

In [14]:
@everywhere function threadexchange(parallel_states,n_threads,idx)
    if rand() < 0.2  #20% change per trajectory of an attempted exchange
        thrid = rand(1:n_threads,2)
        if thrid[1] == thrid[2] && thrid[2] == n_threads
            thrid[2] = rand(1:n_threads-1)
        elseif thrid[1] == thrid[2] && thrid[2] != n_threads
            thrid[2] +=1
        end #which threads are talking

        exc_acc = exc_acceptance(parallel_states[thrid[1]][idx].beta,parallel_states[thrid[2]][idx].beta,parallel_states[thrid[1]][idx].en_tot,parallel_states[thrid[2]][idx].en_tot) #calc acceptance
        
        if exc_acc > rand() #if exchange is likely
            exc_trajectories!(parallel_states[thrid[1]][idx] ,parallel_states[thrid[2]][idx] )#swap
        end
     end
    
end



In [15]:

@everywhere function pptmc_loop!(parallel_states,results,move_strat,mc_params,pot_vector,ensemble,n_steps,a,v,r,save_ham,save,i,delta_en,n_threads)
    @sync begin
        Threads.@threads for i_thread = 1:n_threads
        #run ~1000 steps then attempt exchanges
            for i = 1:500
                ptmc_cycle!(parallel_states[i_thread],results,move_strat,mc_params,pot_vector[i_thread],ensemble,n_steps,a,v,r, save_ham,save,i,delta_en = delta_en)
            end

        end
    end
    #attempt exchanges
    @sync begin
        for idx = 1:mc_params.n_traj
            threadexchange(parallel_states,n_threads,idx)
        end
    end

end

@everywhere function pptmc_run!(mc_states,move_strat,mc_params,pot,ensemble,results; save_ham = false,save = false)
    #we can initialise everything in bulk. 
    parallel_states,pot_vector,a,v,r,delta_en,n_threads = parallel_equilibration(mc_states,move_strat,mc_params,pot,ensemble,results)
    println("equilibration complete")

    num_loops = Int(floor(mc_params.mc_cycles/(500*n_threads) ))
    counter = 0

    for i_loop = 1:num_loops
        pptmc_loop!(parallel_states,results,move_strat,mc_params,pot_vector,ensemble,n_steps,a,v,r,save_ham,save,i,delta_en,n_threads)

        counter += n_threads*500
    end
    println("MC loop done")

    rm("pots",recursive = true)

    #post processing
    en_tot_avg = zeros(mc_params.n_traj)
    en2_tot_avg = zeros(mc_params.n_traj)
    count_stat_atom_tot = zeros(mc_params.n_traj)
    count_stat_exc_tot = zeros(mc_params.n_traj)

    for statevec in parallel_states
        en_avg = [statevec[i_traj].ham[1] / counter  for i_traj in 1:mc_params.n_traj]

        en_tot_avg += en_avg

        en2_avg = [statevec[i_traj].ham[2] / counter  for i_traj in 1:mc_params.n_traj]
        en2_tot_avg += en2_avg

        count_stat_atom = [mc_states[i_traj].count_atom[1] / (mc_params.n_atoms * counter) for i_traj in 1:mc_params.n_traj]
        count_stat_atom_tot += count_stat_atom
        count_stat_exc = [mc_states[i_traj].count_exc[2] / mc_states[i_traj].count_exc[1] for i_traj in 1:mc_params.n_traj]
        count_stat_exc_tot += count_stat_exc


    end
    results.en_avg = en_avg

    results.count_stat_atom,results.count_stat_exc = count_stat_atom_tot,count_stat_exc_tot

    results.heat_cap = [(en2_avg[i]-en_avg[i]^2) * mc_states[i].beta for i in 1:mc_params.n_traj]
    println(results.heat_cap)
    println("done")
    return

end


I want to benchmark the exchanges between threads -- this was done and it is slower to thread the exchange step

Now it's time to test the pptmc loop function to see how quickly we can run 1000*13 atom moves.

In [16]:
 parallel_states,pot_vector,a,v,r,delta_en,n_threads = parallel_equilibration(mc_states,move_strat,mc_params,pot,ensemble,results)


What I need to test is whether writing to and running RuNNer is actually safe from race conditions. This may just be too complicated to manage for Julia

In [9]:
cp("$(pot.dir)/input.data","test.txt")

"test.txt"

In [12]:
function test_time()
    cp("$(pot.dir)/input.data","test.txt", force=true)
end


test_time (generic function with 1 method)

In [13]:
@benchmark test_time()


BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  33.306 μs … 655.080 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     44.943 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   47.934 μs ±  15.931 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▂▄▇▇██▃▃▂▁▁    ▂                                         
  ▁▁▁▁▅█████████████▇▇██▇█▆▄▃▃▃▄▃▃▄▃▃▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  33.3 μs         Histogram: frequency by time         83.3 μs <

 Memory estimate: 2.13 KiB, allocs estimate: 15.

In [14]:
@benchmark getRuNNerenergy(pot.dir , mc_params.n_traj)

BenchmarkTools.Trial: 463 samples with 1 evaluation.
 Range (min … max):   9.747 ms …  21.276 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     10.640 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   10.785 ms ± 848.773 μs  ┊ GC (mean ± σ):  0.15% ± 2.56%

       ▁ ▄▁▄▂▅▂▃▅▃█▅ ▁ ▁                                        
  ▂▃▃▃▆█▇███████████████▇▆▆█▃▄▅▅▃▃▄▄▄▂▃▃▄▁▃▂▃▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▃ ▄
  9.75 ms         Histogram: frequency by time         13.2 ms <

 Memory estimate: 138.05 KiB, allocs estimate: 2714.